## Konfiguracja

plik `build.gradle.kts(Project)
```kotlin
plugins {
    ...
    id("androidx.room") version "2.8.0" apply false
    id("com.google.devtools.ksp") version "2.0.21-1.0.27" apply false
}
```

plik `build.gradle.kts(Module)
```kotlin
plugins {
    ...
    id("androidx.room")
    id("com.google.devtools.ksp")
}

android {
    ...
    room {
        schemaDirectory("$projectDir/schemas")
    }
}

dependencies {

    implementation("androidx.room:room-runtime:2.8.0")
    ksp("androidx.room:room-compiler:2.8.0")
    implementation("androidx.room:room-ktx:2.8.0")

    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.4")
    implementation("androidx.lifecycle:lifecycle-runtime-compose:2.9.4")

    ...
}
```

```kotlin
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            RomBasicsExampleTheme {
                TaskScreen()
            }
        }
    }
}

// =================================================================================
// --- Warstwa Danych: Definicje Room ---
// =================================================================================

// Encja (Tabela w bazie danych)
@Entity(tableName = "tasks")
data class Task(
    @PrimaryKey(autoGenerate = true) val id: Int = 0,
    val title: String,
    val isCompleted: Boolean = false
)

// DAO (Data Access Object) - Interfejs zapytań
@Dao
interface TaskDao {
    @Insert(onConflict = OnConflictStrategy.REPLACE)
    suspend fun insertTask(task: Task)
    @Update
    suspend fun updateTask(task: Task)
    @Delete
    suspend fun deleteTask(task: Task)
    @Query("SELECT * FROM tasks ORDER BY id DESC")
    fun getAllTasksStream(): Flow<List<Task>>
}

// Baza Danych
@Database(entities = [Task::class], version = 1, exportSchema = false)
abstract class AppDatabase : RoomDatabase() {
    abstract fun taskDao(): TaskDao
    companion object {
        @Volatile
        private var INSTANCE: AppDatabase? = null
        fun getDatabase(context: Context): AppDatabase {
            return INSTANCE ?: synchronized(this) {
                val instance = Room.databaseBuilder(
                    context.applicationContext,
                    AppDatabase::class.java,
                    "task_database_db"
                ).build()
                INSTANCE = instance
                instance
            }
        }
    }
}

// Repozytorium - pośrednik między ViewModel a DAO
class TaskRepository(private val taskDao: TaskDao) {
    val allTasks: Flow<List<Task>> = taskDao.getAllTasksStream()
    suspend fun insert(task: Task) {
        taskDao.insertTask(task)
    }
    suspend fun update(task: Task) {
        taskDao.updateTask(task)
    }
}


// =================================================================================
// --- Architektura: ViewModel i Fabryka ---
// =================================================================================

class TaskViewModel(private val repository: TaskRepository) : ViewModel() {
    val tasks: StateFlow<List<Task>> = repository.allTasks
        .stateIn(
            scope = viewModelScope,
            started = SharingStarted.WhileSubscribed(5000),
            initialValue = emptyList()
        )
    fun addTask(title: String) {
        if (title.isNotBlank()) {
            viewModelScope.launch {
                repository.insert(Task(title = title))
            }
        }
    }
    fun toggleTaskCompletion(task: Task) {
        viewModelScope.launch {
            repository.update(task.copy(isCompleted = !task.isCompleted))
        }
    }
}

// Fabryka do tworzenia ViewModelu z zależnością
class TaskViewModelFactory(private val application: Application) : ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        if (modelClass.isAssignableFrom(TaskViewModel::class.java)) {
            val dao = AppDatabase.getDatabase(application).taskDao()
            val repository = TaskRepository(dao)
            @Suppress("UNCHECKED_CAST")
            return TaskViewModel(repository) as T
        }
        throw IllegalArgumentException("Unknown ViewModel class")
    }
}

// =================================================================================
// --- UI ---
// =================================================================================

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun TaskScreen() {
    // Pobranie Application context do fabryki
    val application = LocalContext.current.applicationContext as Application
    val viewModel: TaskViewModel = viewModel(factory = TaskViewModelFactory(application))
    val tasks by viewModel.tasks.collectAsStateWithLifecycle()
    var newTaskTitle by remember { mutableStateOf("") }

    Scaffold(
        topBar = { TopAppBar(title = { Text("Room To-Do Lista") }) }
    ) { padding ->
        Column(modifier = Modifier.padding(padding).fillMaxSize()) {
            // Sekcja dodawania nowego zadania
            Row(
                modifier = Modifier.fillMaxWidth().padding(16.dp),
                verticalAlignment = Alignment.CenterVertically
            ) {
                OutlinedTextField(
                    value = newTaskTitle,
                    onValueChange = { newTaskTitle = it },
                    label = { Text("Nowe zadanie") },
                    modifier = Modifier.weight(1f)
                )
                IconButton(onClick = {
                    viewModel.addTask(newTaskTitle)
                    newTaskTitle = ""
                }) {
                    Icon(Icons.Default.Add, contentDescription = "Dodaj zadanie")
                }
            }
            // Lista zadań
            LazyColumn(modifier = Modifier.weight(1f)) {
                items(tasks) { task ->
                    TaskItem(task = task, onToggle = { viewModel.toggleTaskCompletion(task) })
                }
            }
        }
    }
}

@Composable
fun TaskItem(task: Task, onToggle: () -> Unit) {
    Row(
        modifier = Modifier
            .fillMaxWidth()
            .clickable(onClick = onToggle)
            .padding(horizontal = 16.dp, vertical = 12.dp),
        verticalAlignment = Alignment.CenterVertically
    ) {
        Icon(
            imageVector = if (task.isCompleted) Icons.Default.CheckCircle else Icons.Default.CheckCircle,
            contentDescription = "Status zadania",
            tint = if (task.isCompleted) Color.Green else MaterialTheme.colorScheme.onSurface
        )
        Spacer(Modifier.width(16.dp))
        Text(
            text = task.title,
            style = if (task.isCompleted) TextStyle(textDecoration = TextDecoration.LineThrough) else TextStyle.Default
        )
    }
}
```